# ACRRU RAG Model Testing 
Autonomous Climate Resiliency Research Unit

In [35]:
import pandas as pd
import gspread
import time
from typing import Optional, Type, Union
from enum import Enum

from langchain import hub

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper

from langchain.tools import BraveSearch, BaseTool, StructuredTool, tool

from langchain_openai import ChatOpenAI

from langchain.agents import AgentExecutor, initialize_agent, create_openai_tools_agent, create_self_ask_with_search_agent, AgentType, create_react_agent

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate, chat
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from langchain.pydantic_v1 import BaseModel, Field



## Define some constants 

In [4]:
# Controls how much info is provided during a run
VERBOSE = True

# Controls what OpenAI model to use
OPENAI_MODEL = "gpt-4-turbo-preview"

# Controls degree of randomness in response. Scale is from 0 to 2
MODEL_TEMP = 0.0

# Controls whether to output research plan steps to the user along with final response
RETURN_INTERMEDIATE_STEPS = True

# API key for the Brave web search tool
BRAVE_API_KEY = 'BSAmw298lpAtPuiP5crDrAtOpGPgf5U'

# Filepath to Google Sheets credentials file for automated logging
GSHEETS_ACCT_CRED_PATH = 'C:\\Users\\14102\\Brown\\Internships\\INL\\Auth\\acrru-output-logs-97349d3b2c0b.json'

# Capability Titles for more descriptive logging and tags
capabilities_map = {0: "Capability 1: Governance – Executive Level Engagement and Leadership",
                    1: "Capability 2: Climate-Aware Planning",
                    2: "Capability 3: Stakeholder and Community Collaboration",
                    3: "Capability 4: Resilience and Adaptation Actions",
                    4: "Capability 5: Customer Engagement and Coordination"}

## Utility Functions for Logging Outputs to a Google Sheet

In [5]:
# Connect to google sheets

gc = gspread.service_account(filename=GSHEETS_ACCT_CRED_PATH)

sh = gc.open("ACRRU Output Logs")

def append_to_sheet(data: list, sheet_name: str = "ACRRU Output Logs", tab_name: Union[str, int] = 0) -> None:
    """
    Helper function which appends a list of values to the next available row of a specified Google Sheets tab.
    """
    sheet = gc.open(sheet_name)
    sheet_tab = sheet.get_worksheet(tab_name)

    if isinstance(data[0], list):
        for row in data:
            sheet_tab.append_row(row)
    
    else:
        sheet_tab.append_row(data)


# Collect data to log 
def collect_log_data(agent, agent_output, test_input_dict: dict, target_cap: str, notes: str) -> list:
    """
    Compiles all information about a provided ACRRU research output to be logged into a list. 
    """
    # Get timestamp
    cur_time = time.ctime(time.time())
    # Get intermediate queries the model called itself
    int_steps = '\n\n'.join([int_step[0].log.strip() for int_step in agent_output['intermediate_steps']])
    # Get the input template
    template = '\n'.join([f'{input_value.content}' for input_value in agent.steps[1].format_messages(**test_input_dict, agent_scratchpad=['']) if input_value.__class__.__name__ == 'SystemMessage'])
    # Get the input prompt
    prompt = '\n'.join([f'{input_value.content}' for input_value in agent.steps[1].format_messages(**test_input_dict, agent_scratchpad=['']) if input_value.__class__.__name__ == 'HumanMessage'])

    # Return a list of everything!
    return [cur_time, test_input_dict['org_name'], target_cap, notes, 
           template, prompt, int_steps, agent_output['output']]

## Compile Electricity Providers

Data from [EIA 2022 report](https://www.eia.gov/electricity/data/eia861/)

In [ ]:
# Get list of electricity providers
elec_industry_df = pd.read_csv("C:\\Users\\14102\\Brown\\Internships\\INL\\Data\\Frame_2022.csv")
# Get list of the territory electricity providers service. At the State/County level.
service_territory_df = pd.read_csv("C:\\Users\\14102\\Brown\\Internships\\INL\\Data\\Service_Territory_2022.csv")

service_territory_df = service_territory_df.drop(
    [col for col in service_territory_df.columns if col not in ['Utility Number', 'State', 'County']], axis=1).reset_index(drop=True)

In [ ]:
# Only look at investor owned companies for now!
investor_utils = elec_industry_df[elec_industry_df['Ownership'] == 'Investor Owned']
# Drop some columns
investor_utils = investor_utils.drop(
    [col for col in investor_utils.columns if col not in ['Utility Number', 'Utility Name']], axis=1).reset_index(drop=True)

In [ ]:
# Join investor owned utils with their reported coverage
investor_util_coverage = investor_utils.merge(service_territory_df, on='Utility Number', how='inner')

In [ ]:
investor_util_coverage

## Build Web Searching Tool (Tavily) for Agent

[Tavily link](https://tavily.com/)

[Usage in LangChain](https://python.langchain.com/docs/integrations/tools/tavily_search)

In [6]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results = 10)

In [5]:
# response = tavily_tool.invoke({'query': 'Baltimore Gas and Electric climate resilience executive leadership'})

## Build Backup Search Tool

[Usage in LangChain](https://python.langchain.com/docs/integrations/tools/brave_search)

In [6]:
brave_tool = BraveSearch.from_api_key(api_key=BRAVE_API_KEY, search_kwargs={"count": 4}, description='a search engine. useful to use when other tools do not return results.')

## Get CRMM Text file 

In [7]:
# Full CRMM file with all information
with open('crmm_docs/CRMM_13.txt', "r", encoding='utf-8') as f:
    crmm_raw = f.read()

# Summarized version of CRMM doc (done with ChatGPT)
with open('crmm_docs/CRMM_AI_Summary.txt', "r", encoding='utf-8') as f:
    crmm_condensed = f.read()

# Full CRMM file without the capability section
with open('crmm_docs/CRMM_header.txt', "r", encoding='utf-8') as f:
    crmm_head = f.read()

cap_dict = {}
# CRMM Capability descriptions
for num in range(5):
    with open(f'crmm_docs/Capability {num+1}.txt', "r", encoding='utf-8') as f:
        cap = f.read()
    cap_dict[capabilities_map[num]] = cap

cap_dict['full'] = crmm_raw

## Build Chat Prompt for Agent

[LangChain Documentation](https://python.langchain.com/docs/modules/model_io/prompts/quick_start)

In [8]:
sys_msg_rsrch = SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['crmm_info'], 
                     template="You are an analyst who uses the Climate Resilience Maturity Model to summarize how infrastructure providers are implementing climate resiliency measures. You are very thorough in your research." 
                     "\nYou are critical of providers, and only give them credit for progress in a capability if specific and direct evidence is provided related to infrastructure development or resilience planning and organization." 
                     " The Climate Resilience Maturity Model is explained below:\n####\n{crmm_info}\n"))

sys_msg_agg = SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['crmm_info', 'agg_lvl'], 
                     template="You are an analyst who uses the Climate Resilience Maturity Model to summarize how {agg_lvl}s are implementing climate resiliency measures." 
                     "\nYou are responsible for summarizing information provided to you into a single report for the corresponding {agg_lvl}. You prefer to only use the information provided to you, but will perform additional research if the information seems insufficient." 
                     " The Full Climate Resilience Maturity Model is described below:\n####\n{crmm_info}\n"))

Prompt for all-capability evaluation/research

In [9]:
# OpenAI agents need placeholders for chat history and agent scratchpad.
chat_prompt = ChatPromptTemplate.from_messages(
                # System message gives the model the 'backstory' for the task (eg. how do you act? what info do you have on hand? what are you designed to do?), but does not actually respond to this directly
                [sys_msg_rsrch,
                chat.MessagesPlaceholder(variable_name='chat_history', optional=True),
                # Human message represents the question actually being asked by a user or service. The model's output is a response to this message.
                chat.HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['org_name'], template="Summarize how the infrastructure and initiatives of the organization, {org_name} is positioned with respect to the Climate Resilience Maturity Model. "
                     "For each capability, provide a one sentence summary, list all the evidence that led to the conclusion, and cite sources. All evidence MUST be directly related to the Climate Resilience Maturity Model. Use as many different relevant sources as possible to gauge their overall position. \n"
                     "**Research Approach**: \n 1. Search for any documents related to {org_name} and climate resilience. \n 2. Search for documents specifically related to capabilities not accounted for in step 1. \n"
                     " Do **not** consider the following topics when collecting evidence:\n -greenhouse gas emissions \n -energy saving and efficiency \n -decarbonization \n -clean energy")),
                chat.MessagesPlaceholder(variable_name='agent_scratchpad')])

Prompt for capability-specific evaluation/research

In [10]:
# OpenAI agents need placeholders for chat history and agent scratchpad.
chat_prompt_cap = ChatPromptTemplate.from_messages(
                # System message gives the model the 'backstory' for the task (eg. how do you act? what info do you have on hand? what are you designed to do?), but does not actually respond to this directly
                [sys_msg_rsrch,
                chat.MessagesPlaceholder(variable_name='chat_history', optional=True),
                # Human message represents the question actually being asked by a user or service. The model's output is a response to this message.
                chat.HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['org_name', 'cap_info'], template="Summarize how the infrastructure and initiatives of the organization, {org_name} are positioned with respect to the following capability in the Climate Resilience Maturity Model:\n"
                     "####\n{cap_info}\n####\n"
                     "Answer using the following format:\n"
                     "####\n## Level 1\nSummary: [summary of level 1]\nEvidence: [All links referenced in level 1 summary]\n## Level 2\nSummary: [summary of level 2]\nEvidence: [all links referenced in level 2 summary]\n## Level 3\nSummary: [summary of level 3]\nEvidence: [all links referenced in level 3 summary]" 
                     "\nDirectly mention how each summary relates to the corresponding level criteria. All evidence MUST be related to climate resiliency in some way. Use as many different relevant sources as possible to gauge their overall position. \n"
                     "Do not include an introduction or conclusion in your response. \nDo not consider the following topics when collecting evidence:\n -greenhouse gas emissions \n -energy saving and efficiency \n -decarbonization \n -clean energy")),
                chat.MessagesPlaceholder(variable_name='agent_scratchpad')])

Prompt for provider or region-level aggregation

In [11]:
# OpenAI agents need placeholders for chat history and agent scratchpad.
chat_prompt_agg = ChatPromptTemplate.from_messages(
                # System message gives the model the 'backstory' for the task (eg. how do you act? what info do you have on hand? what are you designed to do?), but does not actually respond to this directly
                [sys_msg_agg,
                chat.MessagesPlaceholder(variable_name='chat_history', optional=True),
                # Human message represents the question actually being asked by a user or service. The model's output is a response to this message.
                chat.HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['org_name', 'summaries'], template="Create a report of the evidence related to the maturity level of {org_name} provided by the following summaries:\n"
                     "######\n{summaries}\n#####\n"
                     "\nProvide one report per capability. If you perform additional research, list the new links you are referencing. Do NOT provide a maturity score. All evidence MUST be related to climate resiliency in some way. Use as many different relevant sources as possible to gauge their overall position. \n"
                     "Do not include an introduction or conclusion in your response.")),
                chat.MessagesPlaceholder(variable_name='agent_scratchpad')])

## Create the Agent

[LangChain Agent Documentation](https://python.langchain.com/docs/modules/agents/quick_start)

[LangChain Tools Documentation](https://python.langchain.com/docs/modules/agents/tools/)

In [12]:
# Define the model (GPT-4)
llm = ChatOpenAI(model=OPENAI_MODEL, temperature=MODEL_TEMP)

In [26]:
# Define tools available to agent
tools = [tavily_tool]

# Use this function to make agents based on OpenAI models
# Define agent for provider-level research
provider_research_agent = create_openai_tools_agent(llm, tools, chat_prompt)

# Define agent for capability-level research
capability_research_agent = create_openai_tools_agent(llm, tools, chat_prompt_cap)

# Define agent for summaries
summary_agent = create_openai_tools_agent(llm, tools, chat_prompt_agg)

In [32]:
# Define agent executor for research
capability_research_agent_executor = AgentExecutor(agent=capability_research_agent, tools=tools, verbose=VERBOSE, return_intermediate_steps=RETURN_INTERMEDIATE_STEPS)
provider_research_agent_executor = AgentExecutor(agent=provider_research_agent, tools=tools, verbose=VERBOSE, return_intermediate_steps=RETURN_INTERMEDIATE_STEPS)

# Define agent executor for research
summary_agent_executor = AgentExecutor(agent=summary_agent, tools=tools, verbose=VERBOSE, return_intermediate_steps=RETURN_INTERMEDIATE_STEPS)

## Running the Agent Iteratively

### Agent run across multiple providers, **assessing all capabilities simultaneously** per organization

In [ ]:
# For an example, lets see what electric utility providers serve DC!
investor_util_coverage[investor_util_coverage['State'] == 'DC']['Utility Name'].unique()

In [ ]:
# Define input list: Currently is investor-owned elec utils from MD
# md_elec_providers = investor_util_coverage[investor_util_coverage['State'] == 'MD']['Utility Name'].unique()
providers = ['Commonwealth Edison Co Illinois']
research_output = {}
# Iterate over every provider in our array
for organization in providers:

    ## Compile Agent Input
    input_dict = {
        'crmm_info': crmm_raw,
        'org_name': organization}

    # Run the agent for given org
    agent_output: dict = provider_research_agent_executor.invoke(input_dict)

    # Record result in dict
    research_output[organization] = agent_output['output']

In [ ]:
rag_output = pd.DataFrame.from_dict(research_output, orient='index', columns=['Research Results']).reset_index(names=['Utility Name'])
rag_output = rag_output.merge(investor_utils, on='Utility Name', how='inner')

## Observing Outputs

In [ ]:
for org, rag_response in research_output.items():
    print(f'RAG Response for {org}:\n')
    print(rag_response)
    print('\n\n\n')

### ACRRU run across one provider, **assessing each capability individually**

In [20]:
# Iterate over all capabilities, and skip over the full crmm item.
state_name = ' Illinois'
org_name = 'Commonwealth Edison co'+ state_name

# Record Agent responses
output_dict = {}

# Custom notes 
notes = "Testing explicit output instruction in prompt, evaluating 10 sources per research step"
for target_cap, info in cap_dict.items():
    # Skip the full CRMM input.
    if target_cap == 'full':
        continue

    # Define input dict for ACRRU
    test_input_dict = {
        # Lets test with a much shorter version of CRMM
    'crmm_info': crmm_condensed, #crmm_head,
    'org_name': org_name,
    'cap_info': info}
    
    # Actually runs ACRRU! Returns a dict of inputs (prompt + template) and output
    agent_output = capability_research_agent_executor.invoke(test_input_dict)
    output_dict[target_cap] = agent_output
    
    # Compile data to log
    log_data = collect_log_data(capability_research_agent, agent_output, test_input_dict, target_cap, notes)

    # Log data in google sheet
    append_to_sheet(data=log_data)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Commonwealth Edison co Illinois executive climate resilience lead'}`


[{'url': 'https://www.power-grid.com/td/comed-to-lead-impact-study-of-climate-change-on-the-grid/', 'content': 'The study, the first in ComEd’s region and believed to be the first that will incorporate the impact of increased electrification into the climate risk planning process, will build upon established climate science and industry best practices to help ComEd plan and build infrastructure that is more resilient to the climate changes that pose growing risks to the grid.\n “Families and businesses in northern Illinois have a front-row seat to the increasingly severe weather caused by climate change, which has brought record-breaking temperature swings, historic tornadoes and hurricane-strength winds that continue to test the resiliency of the power grid and the reliable energy our customers have come to expect,” sai

## Aggregating Research to the Provider-Level

### Collect capability summaries for prompt


In [31]:
cap_sums = []
for cap, data in output_dict.items():
    cap_section = cap + '\n' + data['output']
    cap_sums.append(cap_section)

cap_input = '\n\n\n'.join(cap_sums)
agg_lvl = 'provider'

In [32]:
print(cap_input)

Capability 1: Governance – Executive Level Engagement and Leadership
## Level 3
Summary: Commonwealth Edison Co. (ComEd) demonstrates a high level of executive engagement and leadership in addressing climate resilience, aligning with Level 3 criteria of the Climate Resilience Maturity Model. ComEd's CEO, Gil Quiniones, has been actively involved in leading and advocating for climate resilience initiatives. The company has undertaken a comprehensive Climate Risk and Adaptation Study in partnership with the U.S. Department of Energy's Argonne National Laboratory. This study aims to assess the impact of climate change on the power grid, including risks such as sustained heat and flooding. The initiative is a part of ComEd's broader strategy to ensure the grid's resilience and reliability amidst changing climate conditions. This level of executive involvement and the strategic approach to climate resilience planning and action, including the engagement of a risk committee and the briefing 

### Run Aggregation for one Provider

In [33]:
provider_summary_input_dict = {'crmm_info': crmm_raw,
                               'agg_lvl': agg_lvl,
                               'org_name': org_name,
                               'summaries': cap_input}

agg_output = summary_agent_executor.invoke(provider_summary_input_dict)

# Compile data to log
log_data = collect_log_data(summary_agent, agg_output, provider_summary_input_dict, 'Provider Summary', notes)

# Log data in google sheet
append_to_sheet(data=log_data)



> Entering new AgentExecutor chain...
### Report on Commonwealth Edison Co. (ComEd) Climate Resilience Maturity

#### Capability 1: Governance – Executive Level Engagement and Leadership

Commonwealth Edison Co. (ComEd) exhibits a high level of executive engagement and leadership in climate resilience, aligning with the highest criteria of the Climate Resilience Maturity Model. The CEO, Gil Quiniones, actively leads and advocates for climate resilience initiatives, including a comprehensive Climate Risk and Adaptation Study in partnership with the U.S. Department of Energy's Argonne National Laboratory. This study assesses climate change impacts on the power grid, focusing on risks like sustained heat and flooding. The initiative is part of ComEd's strategy to ensure grid resilience amid changing climate conditions, demonstrating governance and leadership commitment at the highest level of the CRMM.

**Evidence:**
- [ComEd to lead impact study of climate change on the grid](https://w

## Custom Search Tool with Backup (WIP)

[LangChain Documentation](https://python.langchain.com/docs/modules/agents/tools/custom_tools)

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions using the internet."
    args_schema: Type[BaseModel] = SearchInput

    def __init__(self, tavily, brave): 
        super(BaseTool, self).__init__()
        self.tavily = tavily
        self.brave = brave

    def _run(
        self, query: str) -> str:
        """Use the tool."""
        output = self.tavily.run(query)
        if not output:
            output = self.brave.run(query)
        
        return output

    async def _arun(
        self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


In [ ]:
custom_search = CustomSearchTool(tavily_tool, brave_tool)